# 上攻形态的研究

In [1]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
import json

await init_notebook(use_omicron=True)

async def report(key:str, params:dict, result_df:pd.DataFrame):
    """key must be unique, contains a date"""
    await omicron.cache.sys.hmset_dict(key, params)
    
    html = f"""
    <html>
    <head>
        <style>
            table, th, td {{
                border-collapse: collapse;
                border: 0px;
                color: #666;
            }}
        </style>
    </head>
    <body>
    <h2 style="color:#c21">底部上攻</h2>
    {build_table(result_df, "grey_light")}
    </body>
    </html>
    """
    send_html_email(f"底部上攻 {key}", html)

# 放量三连阳

三连阳，且最大量超启动前倍量，prsi处于中位

In [ ]:
async def scan(trigger, n, results, frame_type=FrameType.DAY, tm=None):
    end = arrow.get(tm) if tm else arrow.now()
    start = tf.shift(tf.floor(end, frame_type), -n+1, frame_type)
    for code in Stock.choose(["stock"]):
        sec = Stock(code)
        try:
            bars = await sec.load_bars(start, end, frame_type)
            bars = bars[np.isfinite(bars["close"])]

            trigger(code, bars, results, frame_type)
        except Exception:
            continue
    return results

In [ ]:
def three_red_soldiers(code, bars, results, frame_type):
    bullish = bars["close"] >= bars["open"]
    if np.all(bullish[-3:]):
        v3, v2, v1, v0 = bars["volume"][-4:]
        if np.mean([v2,v1,v0]) < 2 * v3:
            return

        vr = round(np.mean([v2, v1, v0]) / v3, 1)

        rsi = relative_strength_index(bars["close"], 6)[-3:]

        stats = rsiday if frame_type == FrameType.DAY else rsi30
        prsi = [stats.get_proba(code, r) for r in rsi]

        name = Stock(code).display_name
        results.append([name, code, vr, *rsi, *prsi])

In [ ]:
results = []
await scan(three_red_soldiers, 40, results)
df = pd.DataFrame(results, columns=["name", "code", "vr", "rsi2", "rsi1", "rsi0", "prsi2", "prsi1", "prsi0"])

In [ ]:
df[(df.prsi0<0.7)&(df.prsi0>0.5)]

In [2]:
a = [1,4,3,5,2,7,6]
rank(a)

array([0, 3, 2, 4, 1, 6, 5])